In [1]:
#import dependencies

from bs4 import BeautifulSoup as bs
import os
from splinter import Browser
import requests
import pandas as pd
import time
import re

## Step 1 - Scraping

### NASA Mars News

In [2]:
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path':'/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless = False)

In [4]:
news_url = "https://mars.nasa.gov/news/"
browser.visit(news_url)
time.sleep(1)

In [5]:
html = browser.html
soup = bs(html, 'html.parser')

In [6]:
news_title = soup.find('div', class_ = "content_title").text
news_p = soup.find('div', class_ = "rollover_description_inner").text
print(f'news_title: {news_title}')
print(f'news_p: {news_p}')

news_title: Space Samples Link NASA's Apollo 11 and Mars 2020
news_p: While separated by half a century, NASA's Apollo 11 and Mars 2020 missions share the same historic goal: returning samples to Earth.


### JPL Mars Space Images - Featured Image

In [7]:
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_url)
time.sleep(1)

In [8]:
jpl_html = browser.html
jpl_soup = bs(jpl_html, 'html.parser')

In [9]:
img_relative = jpl_soup.find('article')['style']
print(img_relative)
img_relative_link = img_relative.split(" ")[1][5:-3]
print(img_relative_link)


background-image: url('/spaceimages/images/wallpaper/PIA16606-1920x1200.jpg');
/spaceimages/images/wallpaper/PIA16606-1920x1200.jpg


In [10]:
main_url = 'https://www.jpl.nasa.gov'
feature_image_url = main_url + img_relative_link
feature_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA16606-1920x1200.jpg'

### Mars Weather

In [24]:
weather_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(weather_url)

In [25]:
weather_html = browser.html
weather_soup = bs(weather_html, 'html.parser')

In [26]:
mars_weather_texts = weather_soup.find_all("p", class_ = "TweetTextSize TweetTextSize--normal js-tweet-text tweet-text")

for text in mars_weather_texts:
    if text.find(text = re.compile("InSight") ):
        mars_weather_text = text.text
        break
        
mars_weather_link = weather_soup.find('a', class_="twitter-timeline-link u-hidden").text
print(mars_weather_link)
mars_weather = mars_weather_text[0:-len(mars_weather_link)]
print(mars_weather)

pic.twitter.com/f4SheTlQyY
InSight sol 251 (2019-08-11) low -101.0ºC (-149.7ºF) high -26.5ºC (-15.8ºF)
winds from the SSE at 4.1 m/s (9.2 mph) gusting to 17.5 m/s (39.1 mph)
pressure at 7.60 hPa


### Mars Fact

In [14]:
fact_url = "https://space-facts.com/mars/"
browser.visit(fact_url)
time.sleep(1)

In [15]:
fact_html = browser.html
fact_soup = bs(fact_html,"html.parser")

In [16]:
scrape_table = pd.read_html(fact_url)
scrape_table[1]

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [17]:
mars_fact_df = scrape_table[1]
mars_fact_df.rename(columns = {0:'Description', 1:'Value'}, inplace = True)
mars_fact_df = mars_fact_df.set_index('Description')
mars_fact_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [18]:
mars_fact_html = mars_fact_df.to_html()
print(mars_fact_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Value</th>
    </tr>
    <tr>
      <th>Description</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


### Mars Hemispheres

In [19]:
base_hemi_url = "https://astrogeology.usgs.gov"
hemi_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemi_url)
time.sleep(1)
#response = requests.get(hemi_url)
#print(response.text)

In [20]:
hemi_html = browser.html
hemi_soup = bs(hemi_html, 'html.parser')

In [21]:
hemispheres = hemi_soup.find_all('a', class_="itemLink product-item")

In [22]:
hemisphere_dict = []

In [23]:
for result in hemispheres:
    try:
        title = result.find('h3').text
        #print(title)
        link = result['href']
        full_link = base_hemi_url + link
        response = requests.get(full_link)
        soup = bs(response.text, 'html.parser')
        #print(full_link)
        
        try:
            full_img_cl = soup.find('div', class_ = "downloads")
            full_img_href = full_img_cl.find('a')['href']
            hemisphere_dict.append({"title": title, "img_url":full_img_href})
            
        except AttributeError as e:
            print(e)
    except AttributeError as e:
        print(e)
        
        
print(hemisphere_dict)

'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
